In [1]:
pip install tensorflow matplotlib scikit-learn

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/7c/3c/049400232ee2897d613db9009e42269417166c3f8519d46e5c97b6f9e206/tensorflow-2.15.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/46/37/b5e27ab30ecc0a3694c8a78287b5ef35dad0c3095c144fcc43081170bfd6/matplotlib-3.8.2-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/69/8a/cf17d6443f5f537e099be81535a56ab68a473f9393fbffda38cd19899fc8/scikit_learn-1.3.2-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/da/1b/63e20bde0db52c3be7e078b50cf507f4534ad6e47b5e2b01d9ed63bb652f/tensorflow_intel-2.15.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# To improve model generalization, you can perform data augmentation on your images.

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Load and Preprocess Images

In [ ]:
batch_size = 32
image_size = (150, 150)

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'path/to/your/dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Change to 'categorical' if more than two classes
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'path/to/your/dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Change to 'categorical' if more than two classes
    subset='validation'
)

In [ ]:
# Explore Your Data

In [ ]:
sample_images, sample_labels = next(train_generator)

plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(sample_images[i])
    plt.title(f'Class: {sample_labels[i]}')
    plt.axis('off')
plt.show()

In [ ]:
# Step 4: Build and Train Your Model

In [ ]:
# Convolutional Neural Network (CNN):

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile your model, specifying the optimizer, loss function, and metrics:

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model:

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

In [ ]:
# Evaluate your model on a test set:

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

In [ ]:
# Based on the evaluation, fine-tune your model. Adjust hyperparameters, architecture, or collect more data if necessary.

In [ ]:
# Save your trained model for future use

In [ ]:
model.save('bright_flash_detection_model.h5')

In [ ]:
# Convert the Model to TensorFlow SavedModel Format:
# Convert the Keras model to TensorFlow SavedModel format. This format is suitable for serving with TensorFlow Serving.

In [ ]:
loaded_model = tf.keras.models.load_model('bright_flash_detection_model.h5')

In [ ]:
tf.saved_model.save(loaded_model, 'bright_flash_detection_saved_model')

In [ ]:
# TensorFlow Serving is a separate package from TensorFlow. Install it using the following command:

In [ ]:
pip install tensorflow-serving-api

In [ ]:
# Start TensorFlow Serving:
# Open a new terminal in JupyterLab (or another terminal window), and start TensorFlow Serving. Specify the model name and the path to the SavedModel:

In [ ]:
tensorflow_model_server --model_name=bright_flash_detection --model_base_path=/path/to/bright_flash_detection_saved_model


In [ ]:
# Replace /path/to/bright_flash_detection_saved_model with the actual path to your SavedModel

In [ ]:
# Make Predictions in JupyterLab:
# In your Jupyter Notebook, you can use TensorFlow Serving's gRPC API to make predictions. Install the grpcio library:

In [ ]:
pip install grpcio

In [ ]:
# Then, create a Python script to make predictions using the TensorFlow Serving API.
# Replace 'path/to/your/image.jpg' with the actual path to the image you want to classify.

# This example assumes that your model has a signature name of 'serving_default' 
# and that the input tensor is named 'input_image'. Adjust these values based on your model's specifications.

In [ ]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc, get_model_metadata_pb2, predict_pb2
import numpy as np
from PIL import Image

# Function to preprocess an image
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((150, 150))  # Adjust to your model's input size
    img_array = np.array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to make a prediction using TensorFlow Serving
def predict_image(image_path, host='localhost', port=8500):
    # Create gRPC channel
    channel = grpc.insecure_channel(f'{host}:{port}')

    # Create a stub
    stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

    # Prepare request
    request = predict_pb2.PredictRequest()
    request.model_spec.name = 'bright_flash_detection'  # Model name
    request.model_spec.signature_name = 'serving_default'  # Signature name

    # Preprocess the image
    img_array = preprocess_image(image_path)

    # Set input tensor
    request.inputs['input_image'].CopyFrom(tf.make_tensor_proto(img_array, shape=img_array.shape))

    # Make the prediction
    result = stub.Predict(request, timeout=10.0)

    # Extract and print the prediction
    prediction = result.outputs['dense'].float_val[0]
    print(f'Prediction: {prediction}')

# Example usage
predict_image('path/to/your/image.jpg')